In [ ]:
# PDE for Put option
# p_t+0.5*sigma^2*s^2*P_{ss}+r*s*P_{s}-r*P = 0
# Boundary: 
# Now we approximate the solution to the above PDE

In [18]:
# Global Constant
S_list = list(range(0, 50, 10)) + \
           list(range(45, 85, 5)) + \
           list(range(82, 91, 2)) + \
           list(range(91, 110)) + \
           list(range(110, 122, 2)) + \
           list(range(125, 165, 5)) + \
           list(range(170, 210, 10)) + \
           list(range(220, 310, 20)) + \
           list(range(350, 1050, 50)) + \
           list(range(2000, 10001, 1000))

K = 100
T = 0.25
r= 0.1
sigma = 0.8
S_0 = 100
N = 25

In [19]:
import numpy as np
import scipy.stats as si

def european_put(S, K, T, r, sigma):
    """
    Calculate the Black-Scholes price for a European Put Option.

    Parameters:
    S (float): Current stock price
    K (float): Strike price of the option
    T (float): Time to maturity (in years)
    r (float): Risk-free interest rate (annual rate, continuous compounding)
    sigma (float): Volatility of the underlying stock (annual standard deviation of log returns)

    Returns:
    float: Black-Scholes price of the European Put Option
    """
    
    # Calculating d1 and d2
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    
    # Calculate the put price using the Black-Scholes formula
    put_price = K * np.exp(-r * T) * si.norm.cdf(-d2) - S * si.norm.cdf(-d1)
    
    return put_price



put_price = european_put(S_0, K, T, r, sigma)
print(f"The Black-Scholes price of the European put option is: {put_price}")



The Black-Scholes price of the European put option is: 14.451905854467874


In [24]:
# Explicit Scheme

def ExplicitScheme(K,T,S_list,N):
    
    M = len(S_list)
    dtau = T/N
    
    # grid for saving the solutions
    res = [[0 for i in range(N+1)] for j in range(M)]
    #print(len(res[0]))
    for i in range(M):
        res[i][0] = max(K - S_list[i], 0)

    #loop through all the time step
    for n in range(1,N+1,1):
        # boundary condition

        res[0][n] = res[0][n-1]*(1-r*dtau) # lower boundary on s
        res[M-1][n] = 0
        # loop through all the prices s_i
        for i in range(1,M-1,1):

            # use the explicit scheme to update the value with centrel finite difference
            alpha_i = sigma**2*S_list[i]**2/((S_list[i]-S_list[i-1])*(S_list[i+1]-S_list[i]))-r*S_list[i]/(S_list[i+1]-S_list[i-1])
            beta_i = sigma**2*S_list[i]**2/((S_list[i+1]-S_list[i])*(S_list[i]-S_list[i-1]))+r*S_list[i]/(S_list[i+1]-S_list[i-1])

            # avioid scillation:
            
            if alpha_i < 0:
                alpha_i = sigma**2*S_list[i]**2/((S_list[i]-S_list[i-1])*(S_list[i+1]-S_list)[i-1])
                beta_i = sigma**2*S_list[i]**2/((S_list[i+1]-S_list[i])*(S_list[i+1]-S_list[i-1]))+r*S_list[i]/(S_list[i+1]-S_list[i])
            

            res[i][n] = res[i][n-1]*(1-alpha_i+beta_i+r)*dtau + alpha_i*res[i-1][n-1]*dtau + beta_i*res[i+1][n-1]*dtau
    return res

ExplicitScheme(K,T,S_list,200)[27][-1]

1.3842812429866838e+245